In [1]:
from collections import deque, defaultdict, Counter
from heapq import heapify, heappush, heappop
import numpy as np
from copy import deepcopy
import math
import time
from functools import cache, reduce, cmp_to_key
import graphviz
from itertools import product
import matplotlib.pyplot as plt
from bisect import bisect_left, bisect_right
import json

In [2]:
sensors = []
beacons = []

with open("./data/day15.txt") as f:
    while line := f.readline():
        line = line.rstrip().split(': ')
        sensor = tuple(map(lambda x: int(x[2:]), line[0].split('at ')[1].split(', ')))
        sensors.append(sensor)
        beacon = tuple(map(lambda x: int(x[2:]), line[1].split('at ')[1].split(', ')))
        beacons.append(beacon)
len(sensors), sensors[0], beacons[0]

(36, (489739, 1144461), (-46516, 554951))

In [3]:
def dist(point1, point2):
    return sum(map(lambda x: abs(x[0]-x[1]), zip(point1, point2)))

In [4]:
y_target = 2000000

points = set()
for sensor, beacon in zip(sensors, beacons):
    d = dist(sensor, beacon)
    d2 = abs(sensor[1]-y_target)
    for i in range(d-d2+1):
        points.add((sensor[0]+i, y_target))
        points.add((sensor[0]-i, y_target))
points = points.difference(set(beacons))
len(points)

5878678